In [7]:
from ConfigParser import SafeConfigParser
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
from util import save_rdd, get_parser
import json
from operator import add
config = '/home/ellery/wikimedia/missing_articles/missing_articles.ini'
cp = SafeConfigParser()
cp.read(config)

['/home/ellery/wikimedia/missing_articles/missing_articles.ini']

In [12]:
contributions_file = os.path.join(cp.get('general', 'contributions_dir'), 'fr')
names = ["language_code", "user_id", "user", "id","page_title","num_edits","timestamp", "bytes_added"]

def compute_month_diff(x):
    d1 = x[-1]
    d2 = x[-2]
    year1 = int(d1[:4])
    month1 = int(d1[4:6])
    year2 = int(d2[:4])
    month2 = int(d2[4:6])
    
    return (year1*12 + month1) - (year2*12+month2)


    
contributions = sc.textFile(contributions_file)\
.map(get_parser(names))\
.filter(lambda x: len(x) == len(names))\
.map(lambda x: (x['user_id'], x['timestamp']))\
.groupByKey()\
.mapValues(lambda x: sorted(x))\
.filter(lambda x: (len(x[1]) > 1) and (x[1][-1] > '20150500000000'))\
.map(lambda x: (compute_month_diff(x[1]), 1))\
.reduceByKey(add)


In [13]:
monthdist = contributions.collect()

In [17]:
 import pandas as pd
monthdist.sort()
monthdist_df = pd.DataFrame(monthdist)

In [23]:
monthdist_df[5:][1].sum() / float(monthdist_df[1].sum())

0.19424113247003941

In [25]:
monthdist_df[:5][1] / float(monthdist_df[1].sum())

0    0.512185
1    0.158449
2    0.066275
3    0.039250
4    0.029599
Name: 1, dtype: float64

In [26]:
monthdist_df

,0,1
0,0,6368
1,1,1970
2,2,824
3,3,488
4,4,368
5,5,243
6,6,187
7,7,172
8,8,164
9,9,128
